# Clinic Visits Feature Table
**Author:** Marang Mutloatse

**Description:** This script performs feature creation on the clinic visits table.

**Version:** 0.0.1

**Status:** Development

## Import Libraries

In [1]:
import os
# get parent user path
user_dev_path = os.path.dirname(os.getcwd())
# get parent path for package
package_path = os.path.dirname(user_dev_path)
import sys
sys.path.insert(0,package_path + '/src/')
import pandas as pd
import datetime as dt
import warnings
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import math
import glob
import yaml
from hiv_support_package import visit_features,target_creation,helper_functions
from datetime import timedelta
plt.style.use('fivethirtyeight')
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

## Working Functions

In [2]:
def load_excel_sheet(path: str):
    if path is not None:
        try:
            xls = pd.ExcelFile(path)
            df = xls.parse(skiprows=0)
            return df
        except Exception as e:
            print(f"Exception on loading excel spreadsheet with error: {e}")

## Loading data

In [3]:
 try: 
    with open (user_dev_path + '/' + 'config_LTFU.yaml', 'r') as file:
        config = yaml.safe_load(file)
except Exception as e:
    print(f'Error reading the config file, {e}')

In [4]:
input_root = config['rise_files']['raw_data_path']
clinic_file = config['rise_files']['raw_clinic_file']
clinic_input = input_root + clinic_file
print(clinic_input)

/data/rise_data/ClinicData.xlsx


In [5]:
df = load_excel_sheet(clinic_input)

### Initial Data Check

In [6]:
helper_functions.print_num_rows_cols(df,'Clinic')

Clinic data has 38 columns and 924862 rows.


In [7]:
helper_functions.check_null_values(df)

There are 22 columns that have missing values


Missing Values  % of Total Values
covid_confirmed               924862              100.0
GESTATIONAL_AGE               924862              100.0
STI_IDS                       924862              100.0
STI_TREATED                   924862              100.0
date_covid_confirmed          924862              100.0
ADR_IDS                       924810              100.0
OI_IDS                        923912               99.9
ADHERE_IDS                    923328               99.8
LMP                           921987               99.7
MATERNAL_STATUS_ART           917313               99.2
offered_index                 843825               91.2
REGIMEN                       775986               83.9
REGIMENTYPE                   775890               83.9
NOTES                         760323               82.2
BP                            467247               50.5
ADR_SCREENED                  387249               41.9
OI_SCREENED                   383469               41.5
ADHERENCE_LEVEL               375454               40.6
CLINIC_STAGE                  190872               20.6
tb_status                      61534                6.7
func_status                    28484                3.1
NEXT_APPOINTMENT               26099                2.8

In [8]:
drop_cols = ["covid_confirmed",
             "GESTATIONAL_AGE",
             "STI_IDS",
             "STI_TREATED",
             "date_covid_confirmed",
             "ADR_IDS",
             "OI_IDS",
             "ADHERE_IDS",
             "LMP",
             "MATERNAL_STATUS_ART",
             "offered_index",
             "REGIMEN",
             "REGIMENTYPE",
             "NOTES"]
df.drop(columns=drop_cols,inplace=True)

## Rename Columns Names

In [9]:
rename_cols = {"name":"STATE","name.1":"L.G.A","name.2":"FACILITY_NAME"}
df = df.rename(columns=rename_cols)

## Filtering Data

In [10]:
pd.set_option('display.max_columns', 500)
df = df.sort_values(by=['PATIENT_ID','DATE_VISIT'])

In [11]:
print("Shape before cleaning by date: ")
helper_functions.print_num_rows_cols(df,'Clinic')

date_cols = ['DATE_VISIT','NEXT_APPOINTMENT']

df['DATE_VISIT'] = pd.to_datetime(df['DATE_VISIT'], errors='coerce')
df['NEXT_APPOINTMENT'] = pd.to_datetime(df['NEXT_APPOINTMENT'], errors='coerce')

df = df.loc[(df['DATE_VISIT'] >= '2010-01-01 00:00:00') & (df['DATE_VISIT'] <= '2021-05-31 00:00:00')]

print(" ")
print("Shape after limiting by date: ")
helper_functions.print_num_rows_cols(df,'Clinic')

Shape before cleaning by date: 
Clinic data has 24 columns and 924862 rows.
 
Shape after limiting by date: 
Clinic data has 24 columns and 908110 rows.


In [12]:
df = df[~df['DATE_VISIT'].isnull()]
print("df with all next appointments:",df.shape)
df.loc[df['NEXT_APPOINTMENT'].isnull(),'NEXT_APPOINTMENT'] = df.DATE_VISIT+timedelta(days=30)
print("df with NULL next appointments filled:",df.shape)

df with all next appointments: (908110, 24)
df with NULL next appointments filled: (908110, 24)


## Inclusion Criteria

In [13]:
start_date = '2010-01-01 00:00:00'
LAST_EXPECTED_VISIT = '2021-05-31 00:00:00' 

In [14]:
print("PATIENT_ID: " + "{:,}".format(df.PATIENT_ID.nunique()))

PATIENT_ID: 128,749


# Data Cleaning

## Deduplication

In [15]:
print(f"Before deduplication the dataframe has {df.shape[0]} rows")
print(" ")
df.drop_duplicates(subset=['PATIENT_ID','DATE_VISIT','FACILITY_ID'],inplace=True, keep="last")
print(f"After deduplication the dataframe has {df.shape[0]} rows")

Before deduplication the dataframe has 908110 rows
 
After deduplication the dataframe has 902609 rows


## Categories Cleaning

In [16]:
# standardize entries in columns
standardize_col = ['func_status', 'tb_status']
for col in standardize_col:
    df[col] = df[col].str.title()

In [17]:
#map incorrect entries to same actual values
clinic_stage_dict = {"Stage I": "Stage I",
                     "Stage II": "Stage II",
                     "Stage III": "Stage III",
                     "Stage IV": "Stage IV",
                     "Stage II?": "Stage III",
                     "Stage IIo": "Stage III",
                     "Stage IId": "Stage III",
                     "Stage IIn": "Stage III",
                     "Stage IIe": "Stage III",
                     "Stage IIt": "Stage III",
                     "Stage III": "Stage III",
                     "Stage IIk": "Stage III"}

func_status_dict = {"Working": "Working",
                    "Asymptomatic": "Asymptomatic",
                    "Ambulatory": "Ambulatory",
                    "Asymptomatic,Asymptomatic": "Asymptomatic",
                    "Bedridden": "Bedridden",
                    "Bed Ridden": "Bedridden",
                    "Symptomatic": "Symptomatic",
                    "Symptomatic,Symptomatic": "Symptomatic"}

In [18]:
df['CLINIC_STAGE'] = df['CLINIC_STAGE'].map(clinic_stage_dict)
df['func_status'] = df['func_status'].map(func_status_dict)
#df['tb_status'] = df['tb_status'].map(tb_status_dict)

## Missing Value Imputation

In [19]:
df['CLINIC_STAGE'] = df.groupby('PATIENT_ID')['CLINIC_STAGE'].transform(
    lambda x: x.ffill().bfill()
)
df['func_status'] = df.groupby('PATIENT_ID')['func_status'].transform(
    lambda x: x.ffill().bfill()
)

df['tb_status'] = df.groupby('PATIENT_ID')['tb_status'].transform(
    lambda x: x.ffill().bfill()
)

df['OI_SCREENED'] = df['OI_SCREENED'].fillna("Unspecified")
df['ADHERENCE_LEVEL'] = df['ADHERENCE_LEVEL'].fillna("Unspecified")
df['ADR_SCREENED'] = df['ADR_SCREENED'].fillna("Unspecified")

df.loc[(df['CLINIC_STAGE'].isnull()) & ((df['func_status'] == 'Asymptomatic') | (df['func_status'] == 'Working')), 'CLINIC_STAGE'] = 'Stage I'
df.loc[(df['CLINIC_STAGE'].isnull()) & (df['func_status'] == 'Ambulatory') , 'CLINIC_STAGE'] = 'Stage II' 
df.loc[(df['CLINIC_STAGE'].isnull()) & (df['func_status'] == 'Bedridden') | (df['func_status'] == 'Symptomatic') , 'CLINIC_STAGE'] = 'Stage III'
df.loc[(df['CLINIC_STAGE'].isnull()) & (df['func_status'].isnull()), 'CLINIC_STAGE'] = 'Stage I'

In [20]:
df["func_status"] = df.groupby(["STATE", "L.G.A", "FACILITY_NAME","CLINIC_STAGE"], sort=False)[
    "func_status"
].apply(lambda x: x.fillna(x.mode().iloc[0]))

df["tb_status"] = df.groupby(["STATE", "L.G.A", "FACILITY_NAME","CLINIC_STAGE"], sort=False)[
    "tb_status"
].apply(lambda x: x.fillna(x.mode().iloc[0]))

## Encode Categorical Values

In [21]:
clinic_stage_map = {"Stage I": 1,
                     "Stage II": 2,
                     "Stage III": 3,
                     "Stage IV": 4}

func_status_map = {"Working":1,
                 "Asymptomatic":1,
                 "Ambulatory":2,
                 "Symptomatic":3,
                 "Bedridden":3}

adherence_level_map = {"Good":1,
                      "Unspecified":1,
                      "Fair":2,
                      "Good":3}

oi_screened_map = {"No":0,
                  "Yes":1,
                  "Unspecified":0}

adr_screened_map = {"No":0,
                   "Yes":1,
                   "Unspecified":0}

In [22]:
df['CLINIC_STAGE'] = df['CLINIC_STAGE'].map(clinic_stage_map)
df['func_status'] = df['func_status'].map(func_status_map)
df['ADHERENCE_LEVEL'] = df['ADHERENCE_LEVEL'].map(adherence_level_map)
df['OI_SCREENED'] = df['OI_SCREENED'].map(oi_screened_map)
df['ADR_SCREENED']  = df['ADR_SCREENED'].map(adr_screened_map)

## Feature Engineering

In [23]:
# Clinic Stage worsened if 
df['STAGE_WORSENED'] = df.groupby('PATIENT_ID')['CLINIC_STAGE'].diff().ge(1).astype(int)

In [24]:
# Change 0.0 kg body weight measurement to previous reading
df['BODY_WEIGHT'] = (df['BODY_WEIGHT'].where(df['BODY_WEIGHT'].ne(0))
                                  .groupby(df['PATIENT_ID'])
                                  .transform(lambda x: x.ffill().bfill())
                                  .fillna(0))

# Deduplicated incorrectly updated field
df = df[(df.groupby('PATIENT_ID')
      ['DATE_VISIT']
      .diff(-1).dt.days.mul(-1) # calculate the difference
      .fillna(float('inf'))     # make sure last row is kept
      .ge(4)                    # select diff >= 4
   )]

# Weight Loss 
df['WEIGHT_LOSS'] = (df.groupby('PATIENT_ID')
                       ['BODY_WEIGHT']
                       .pct_change() # calculate percent change
                       .le(-0.1)     # loss if lower than -0.1 (-10%)
                       .astype(int)  # convert True/False to 1/0
                     )

# Date Featurization

In [25]:
df['EF_DAYS_UNTIL_NEXT_APPOINTMENT'] = (df['NEXT_APPOINTMENT'] - df['DATE_VISIT']).dt.days

In [26]:
# Generate Date Features
df = visit_features.generate_date_features(df,'DATE_VISIT')
df = visit_features.generate_date_features(df,'NEXT_APPOINTMENT')

Generating features for DATE_VISIT, starting with  rows:802,298; cols: 27
Generating features for DATE_VISIT,ending with  rows:802,298; cols: 39
Generating features for NEXT_APPOINTMENT, starting with  rows:802,298; cols: 39
Generating features for NEXT_APPOINTMENT,ending with  rows:802,298; cols: 51


In [27]:
# Get first visit day
df['EF_FIRST_DATE_VISIT'] = df.groupby('PATIENT_ID')['DATE_VISIT'].transform('min')
# generate sequence based off patient ID
df['EF_ID_SEQUENCE'] = df.groupby('PATIENT_ID').cumcount() + 1

In [28]:
df = df.loc[(df['EF_FIRST_DATE_VISIT'] > start_date)]
print("Shape after filtering early starters before: ", start_date, visit_features.format_shape(df))

Shape after filtering early starters before:  2010-01-01 00:00:00 rows:802,224; cols: 53


In [29]:
bins_NextAppDays = [-100,-2,0,1,32,100,420,10000]
a = df['EF_DAYS_UNTIL_NEXT_APPOINTMENT'].value_counts(bins=bins_NextAppDays).sort_index()
b = df['EF_DAYS_UNTIL_NEXT_APPOINTMENT'].value_counts(bins=bins_NextAppDays, 
                                                          normalize=True).sort_index()
b = (b*100).round(0)
pd.concat([a,b], axis=1)

EF_DAYS_UNTIL_NEXT_APPOINTMENT  \
(-100.001, -2.0]                            1142   
(-2.0, 0.0]                                 1794   
(0.0, 1.0]                                   469   
(1.0, 32.0]                               363357   
(32.0, 100.0]                             321319   
(100.0, 420.0]                            107842   
(420.0, 10000.0]                            4033   

                  EF_DAYS_UNTIL_NEXT_APPOINTMENT  
(-100.001, -2.0]                             0.0  
(-2.0, 0.0]                                  0.0  
(0.0, 1.0]                                   0.0  
(1.0, 32.0]                                 45.0  
(32.0, 100.0]                               40.0  
(100.0, 420.0]                              13.0  
(420.0, 10000.0]                             1.0

In [30]:
print("All: "+ format(len(df.index), ','))

print("More than 1 days: "+ format(len(df[df['EF_DAYS_UNTIL_NEXT_APPOINTMENT'] > 1].index), ','))
print("More than 32 days: "+ format(len(df[df['EF_DAYS_UNTIL_NEXT_APPOINTMENT'] > 32].index), ','))
print("More than 100 days: "+ format(len(df[df['EF_DAYS_UNTIL_NEXT_APPOINTMENT'] > 100].index), ','))
print("More than 420 days: "+ format(len(df[df['EF_DAYS_UNTIL_NEXT_APPOINTMENT']> 420].index), ','))

print("Less than 1 days: "+ format(len(df[df['EF_DAYS_UNTIL_NEXT_APPOINTMENT'] < 1].index), ','))
print("Less than 10 days: "+ format(len(df[df['EF_DAYS_UNTIL_NEXT_APPOINTMENT'] < 10].index), ','))
print("Less than 32 days: "+ format(len(df[df['EF_DAYS_UNTIL_NEXT_APPOINTMENT'] < 32].index), ','))
print("Less than 100 days: "+ format(len(df[df['EF_DAYS_UNTIL_NEXT_APPOINTMENT'] < 100].index),','))
print("Less than 420 days: "+ format(len(df[df['EF_DAYS_UNTIL_NEXT_APPOINTMENT'] < 420].index),','))

All: 802,224
More than 1 days: 796,660
More than 32 days: 433,303
More than 100 days: 111,984
More than 420 days: 4,142
Less than 1 days: 5,095
Less than 10 days: 8,676
Less than 32 days: 365,182
Less than 100 days: 689,935
Less than 420 days: 798,039


In [31]:
df = visit_features.days_months_since_visit_features(df)
df = df.rename(columns={'NEXT_APPOINTMENT': 'EF_NEXT_APPOINTMENT'})
#df = visit_features.next_actual_visit_features(df)
df = visit_features.months_first_visit_features(df)

In [32]:
df['EF_NEXT_ACTUAL_VISIT_DATE'] = df.groupby('PATIENT_ID')['DATE_VISIT'].shift(-1)
df['EF_DAYS_BETWEEN_NEXT_APPOINTMENT_AND_ACTUAL_VISIT_DATE'] = df['EF_NEXT_ACTUAL_VISIT_DATE'] - df['EF_NEXT_APPOINTMENT']

In [33]:
# Classic LTFU Pepfar Target
df = target_creation.missed_visit_target(df,LAST_EXPECTED_VISIT)

# General Targets
df = target_creation.next_visit_target(df,LAST_EXPECTED_VISIT)
df = target_creation.last_visit_target(df,LAST_EXPECTED_VISIT)

# Rise target
#df = target_creation.rise_missed_visit_target(df,LAST_EXPECTED_VISIT)

In [34]:
df = visit_features.days_late_features(df)
# Indicator Based on appointment
df['EF_LATE_THIS_VISIT'] = df['EF_DAYS_LATE'].dt.days>0

# Late Last visit features 
df = visit_features.late_last_visit_feature(df)

In [35]:
dist = df['EF_DAYS_BETWEEN_NEXT_APPOINTMENT_AND_ACTUAL_VISIT_DATE'
  ].dt.days.sort_values().value_counts(normalize=True, bins=[-400,-20,0,30,90,3000]).sort_index()
print(dist)
print(df['EF_LATE_THIS_VISIT'].value_counts(normalize=True))

(-400.001, -20.0]    0.041930
(-20.0, 0.0]         0.273314
(0.0, 30.0]          0.271875
(30.0, 90.0]         0.142132
(90.0, 3000.0]       0.106291
Name: EF_DAYS_BETWEEN_NEXT_APPOINTMENT_AND_ACTUAL_VISIT_DATE, dtype: float64
True     0.521807
False    0.478193
Name: EF_LATE_THIS_VISIT, dtype: float64


In [36]:
# Based on next appointment date
df = visit_features.late_days_count_ratio_features(df, 3,'PATIENT_ID')
df = visit_features.late_days_count_ratio_features(df, 28,'PATIENT_ID')
df = visit_features.late_days_count_ratio_features(df, 60,'PATIENT_ID')
df = visit_features.late_days_count_ratio_features(df, 90,'PATIENT_ID')

In [37]:
df['EF_FACILITY_ID'] = df['FACILITY_ID']
df = visit_features.create_last_facility_codes_EF(df,'PATIENT_ID')

In [38]:
# Counts number of Facilties
df['EF_VISITS_AT_THIS_FACILITY'] = df.groupby(['PATIENT_ID','FACILITY_ID'])['CLINIC_ID'].cumcount()+1
#df['EF_FACILITY_COMMITMENT'] = (df['EF_VISITS_AT_THIS_FACILITY']/df['EF_ID_SEQUENCE']).round(1)

# Regimen count on current regimen
# df['EF_VISITS_ON_THIS_REGIMEN'] = df.groupby(['PATIENT_ID','Regimen'])['DATE_VISIT'].cumcount()+1
# df['EF_REGIMEN_COMMITMENT'] = (df['EF_VISITS_ON_THIS_REGIMEN']/df['EF_ID_SEQUENCE']).round(1)

In [39]:
df['EF_DAYS_BETWEEN_NEXT_APPOINTMENT_AND_ACTUAL_VISIT_DATE'] = df['EF_DAYS_BETWEEN_NEXT_APPOINTMENT_AND_ACTUAL_VISIT_DATE']/np.timedelta64(1,'D')
df['EF_DAYS_LATE'] = df['EF_DAYS_LATE']/np.timedelta64(1,'D')

## Push to Data Folder

In [40]:
df = df.add_prefix('C_')
#df = df.add_suffix('_CLINIC')

In [41]:
df.columns =df.columns.str.upper()

In [42]:
df.to_csv(input_root+'feature_clinic.csv',index=False,header=True)